In [ ]:
import torch
import torch.optim as optim
import numpy as np
import time
from tqdm import tqdm

from torch.utils.data import DataLoader,Dataset
from datasets import Robot_Traj_Dataset_Experiment_Real_World_1

import matplotlib.pyplot as plt
from scipy.spatial import distance
%matplotlib inline

from losses import task_loss_with_deform, task_loss
from networks import NFSMP_Implicit, NFSMP_Implicit_Inference
from visualize import img_predict, img_predict_inference, img_predict_interp, img_predict_interp_plot, contour_plot, contour_plot_interp
from evaluate import validate_real_1

import pickle
import cv2
import copy

import numpy as np
import torch
from os.path import join

import trimesh
import glob
import open3d as o3d

train = False

In [ ]:
traj_count = 18
dataset = Robot_Traj_Dataset_Experiment_Real_World_1(traj_count)
dataloader = DataLoader(dataset, shuffle=True, batch_size=1, num_workers=0, drop_last = True)
fig,ax=plt.subplots(3,3,figsize=(18,18))
for i in range(9):
    ax[i//3,i%3].imshow(dataset.demo_img[i]/255.0)
plt.show()
# Shows trajectory samples
cm = plt.cm.get_cmap('RdYlBu')
plt.figure(figsize=(16,16))
mask = dataset.df[0]<1
plt.scatter(dataset.xy[0,:,2][mask],dataset.xy[0,:,1][mask],c = dataset.df[0][mask],cmap=cm)

In [ ]:
if train:
    total_steps=0
    epochs=1200
    lowest_mp_loss = 1000
    for seed in range(10):
        model = NFSMP_Implicit(traj_count//2)
        model.to(device=torch.device('cuda:0'))
        optim = torch.optim.Adam([
                        {'params': model.mp_net.parameters()},
                        {'params': model.sp_net.parameters()},
                        {'params': model.deform_net_sp.parameters()},
                        {'params': model.deform_net_mp.parameters()},
                        {'params': model.hyper_net_mp_deform.parameters()},
                        {'params': model.hyper_net_sp_deform.parameters()},
                        {'params': model.latent_codes.parameters(),'lr':1e-4},
                    ],
            lr=1e-3)
        with tqdm(total = len(dataloader) * epochs) as pbar:
            train_losses = []
            for epoch in range(epochs):
                if epoch%200==199:
                    img_predict_interp_plot(model,128,epoch,0,8)
                    contour_plot_interp(model,epoch=epoch,index1=0,index2=8)
                model.train()
                for step, (model_input, gt) in enumerate(dataloader):

                    start_time = time.time()
                    model_input = {key: value.cuda() for key, value in model_input.items()}
                    gt = {key: value.cuda() for key, value in gt.items()}

                    losses = model(model_input,gt,epoch)

                    train_loss = 0.
                    for loss_name, loss in losses.items():
                        single_loss = loss.mean()

                        if epoch %100== 0 and step==0:
                            print(loss_name,single_loss.item()  )
                        train_loss += single_loss            

                    train_losses.append(train_loss.item())
                    optim.zero_grad()
                    train_loss.backward()
                    optim.step()
                    pbar.update(1)
                    pbar.set_postfix(loss=train_loss.item(), time=time.time() - start_time, epoch=epoch)
                    total_steps += 1
                if epoch>1000 and epoch%20==0:
                    curr_error = validate_real_1(model)
                    if curr_error<lowest_mp_loss:
                        print(curr_error,epoch)
                        lowest_mp_loss=curr_error
                        checkpoint = copy.deepcopy(model.state_dict())
else:
    model = NFSMP_Implicit(traj_count//2)
    model.to(device=torch.device('cuda:0'))
    model.load_state_dict(torch.load("real_exp_1"))
    model.eval()

In [ ]:
if train:
    model.load_state_dict(checkpoint)
    torch.save(model.state_dict(), "real_exp_1")
    print(lowest_mp_loss)
    model.eval()

In [ ]:
# contour_plot_interp(model,epoch=1000,index1=0,index2=6)
# contour_plot_interp(model,epoch=1000,index1=0,index2=2)
contour_plot_interp(model,epoch=1000,index1=8,index2=6)

In [ ]:
# img_predict_interp_plot(model,256,1000,0,6)
# img_predict_interp_plot(model,256,1000,0,2)
img_predict_interp_plot(model,256,1000,0,8)

In [ ]:
from networks import NFSMP_Implicit_Inference
test_model = NFSMP_Implicit_Inference(9,model)
test_model.to(device=torch.device('cuda:0'))
optim = torch.optim.Adam([
                {'params': test_model.latent_codes.parameters()},
            ],
    lr=0.1)
val_dataloader = DataLoader(dataset, shuffle=False, batch_size=9, num_workers=0, drop_last = True)

In [ ]:
from tqdm import tqdm
import time
%matplotlib inline

total_steps=0
epochs=200
with tqdm(total = len(val_dataloader) * epochs) as pbar:
    train_losses = []
    for epoch in range(epochs):
        if epoch%200==199:
            img_predict(test_model,epoch=epoch*5)
        test_model.train()
        for step, (model_input, gt) in enumerate(val_dataloader):

            start_time = time.time()
            model_input = {key: value.cuda() for key, value in model_input.items()}
            gt = {key: value.cuda() for key, value in gt.items()}
            losses = test_model(model_input,gt,epoch*5)
            train_loss = 0.
            for loss_name, loss in losses.items():
                single_loss = loss.mean()
                if epoch %100== 0 and step==0:
                    print(loss_name,single_loss)
                train_loss += single_loss            

            train_losses.append(train_loss.item())
            optim.zero_grad()
            train_loss.backward()
            optim.step()
            pbar.update(1)
            pbar.set_postfix(loss=train_loss, time=time.time() - start_time, epoch=epoch)
            total_steps += 1


In [ ]:
import matplotlib.patches as patches

fig,axs =plt.subplots(1,1,figsize=(12,4))
t_condition = 0

t_corr = int((t_condition+0.5)//(1/100))

index1=2
index2=0

upper_mode = False
lower_mode = True

for p in range(6):
    demo_x = np.linspace(-0.5,0.5,100).reshape(-1,1)
    demo_y = np.zeros((100,3))
    if upper_mode:
        demo_y[:(t_corr+1)] = (dataset.y[0,t_corr,:]-dataset.y[0,0,:])/(t_corr+1)    
#         demo_y[(t_corr+1):] = (dataset.y[0,99,:]-dataset.y[0,t_corr,:])/(t_corr+1)    
    elif lower_mode:
        demo_y[:(t_corr+1)] = (dataset.y[0,100+t_corr,:]-dataset.y[0,100,:])/(t_corr+1)
#         demo_y[(t_corr+1):] = (dataset.y[0,199,:]-dataset.y[0,100+t_corr,:])/(t_corr+1)    
    x_ = torch.from_numpy(demo_x).cuda().float().reshape(-1,1)
    x_.requires_grad=False
    y_dot = torch.from_numpy(demo_y).cuda().float()
    y_dot.requires_grad=True
    
    optim = torch.optim.Adam([y_dot,], lr=3e-4)
    
    for i in range(50):
        y_ = torch.zeros(100,3).cuda()
        y_+= torch.from_numpy(dataset.y[0,0,:]).float().cuda()
        for t in range(99):
            y_[t+1] = y_[t] + y_dot[t]
        samples = torch.cat([x_,y_],dim=1)

        subject_idx = torch.Tensor([index1]).squeeze().long().cuda()[None,...]
        subject_idx2 = torch.Tensor([index2]).squeeze().long().cuda()[None,...]
        embedding = model.latent_codes(subject_idx)*(1-p/5.0)+model.latent_codes(subject_idx2)*p/5.0 
        model_in = {'coords': samples}
        hypo_params_mp = model.hyper_net_mp_deform(embedding)
        deform =model.deform_net_mp(model_in,1000, params=hypo_params_mp)['model_out']  
        model_in = {'coords': samples+deform}
        mp  = model.mp_net(model_in,1000)['model_out']

        z = mp
        
#         y_smoothness = 3e3*torch.mean((y_dot[1:-3]-y_dot[:-4])**2)
        z_constraint = torch.mean(torch.abs(z[:i*4]))  
        loss = z_constraint
        optim.zero_grad()
        loss.backward()
        optim.step()

    xy = samples.detach().cpu().numpy()

    axs.plot(xy[:99,2],xy[:99,1],color = cm(p*25),lw=3)    
    
if upper_mode:
    axs.plot(dataset.y[index1,:100,1],dataset.y[index1,:100,0],color = 'darkred',lw=6,linestyle='--',label='Demonstration 0')
    axs.plot(dataset.y[index2,:100,1],dataset.y[index2,:100,0],color = 'orange',lw=6,linestyle='--',label='Demonstration 2')
elif lower_mode:
    axs.plot(dataset.y[index1,100:200,1],dataset.y[index1,100:200,0],color = 'darkred',lw=6,linestyle='--',label='Demonstration 0')
    axs.plot(dataset.y[index2,100:200,1],dataset.y[index2,100:200,0],color = 'orange',lw=6,linestyle='--',label='Demonstration 2')

plt.legend()
plt.ylim([0.35,-0.05])
rect = patches.Rectangle((-0.20, 0.10), 0.20, 0.10, linewidth=1, edgecolor='black', facecolor='gray')
axs.add_patch(rect)
plt.show()

# axs.plot(dataset.y[index2,100:200,1],dataset.y[index2,100:200,0],color = 'orange',lw=6)

# Different Visualizations

## Interpolation Videos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from functools import partial
import matplotlib.patches as patches
writer = animation.FFMpegWriter(
    fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
### RGB Interpolation Video Creation
xlist = np.linspace(-0.5, 0.5, 1280)
ylist = np.linspace(-0.5, 0.5, 720)
X, Y = np.meshgrid(xlist, ylist)

fig, ax = plt.subplots()
line1, = ax.plot([], [], 'ro')
Z_rgb = list()

index_pairs = [(0,2),
               (2,8),
               (8,6)]
for pair in index_pairs:
    for index in range(30):
        with torch.no_grad():
            samples = torch.from_numpy(np.vstack([Y.reshape(-1),X.reshape(-1)])).float().permute(1,0).cuda()            
            samples.requires_grad=False

            subject_idx = torch.Tensor([pair[0]]).squeeze().long().cuda()[None,...]
            embedding1 = model.latent_codes(subject_idx)    
            subject_idx = torch.Tensor([pair[1]]).squeeze().long().cuda()[None,...]
            embedding2 = model.latent_codes(subject_idx)    
            embedding = embedding1*(1-index/29.0)+embedding2*(index/29.0)
            out = model.inference(samples,embedding,epoch=1000)['rgb'].squeeze().detach().cpu().numpy()+0.5
            out[out>1]=1
            out[out<0]=0
            Z_rgb.append(out.reshape(720,1280,3))
fig,ax=plt.subplots(1,1,figsize=(64,36))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False) 
ims = []
xlist = np.linspace(-0.3, 0.55, 32)
ylist = np.linspace(-0.5, 0.35, 32)
for i in range(len(Z_rgb)):
    im = ax.imshow(Z_rgb[i], animated=True)
    ims.append([im])
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)
ani.save("exp3_rgb.mp4", writer=writer)            

In [ ]:
## Cost Function Interpolation Video Creation

xlist = np.linspace(-0.3, 0.55, 32)
ylist = np.linspace(-0.5, 0.35, 32)
zlist = np.linspace(0, 0.4, 32)
tlist = np.linspace(-0.5, 0.5, 32)      
X,T,   Y, Z = np.meshgrid(xlist,tlist,ylist,zlist)
Z_mp = list() #y.reshape(64,64)

index_pairs = [(0,2),
               (2,8),
               (8,6)]
for pair in index_pairs:
    for index in range(30):
        with torch.no_grad():
            samples = torch.from_numpy(np.vstack([Y.reshape(-1),X.reshape(-1)])).float().permute(1,0).cuda()            
            samples.requires_grad=False

            subject_idx = torch.Tensor([pair[0]]).squeeze().long().cuda()[None,...]
            embedding1 = model.latent_codes(subject_idx)    
            subject_idx = torch.Tensor([pair[1]]).squeeze().long().cuda()[None,...]
            embedding2 = model.latent_codes(subject_idx)    
            embedding = embedding1*(1-index/29.0)+embedding2*(index/29.0)
            samples_mp = torch.from_numpy(np.vstack([T.reshape(-1),X.reshape(-1),Y.reshape(-1),Z.reshape(-1)])).float().permute(1,0).cuda()
            samples_mp.requires_grad=False
            out = model.inference_mp(samples_mp,embedding,epoch=1000)  
            z_mp = out['mp'].squeeze().detach().cpu().numpy() 
            z_mp[z_mp>0.9]=0.9
            z_mp[z_mp<0]=0
            z_mp = z_mp.reshape(32,32,32,32)
            Z_mp.append(z_mp)


# Create a figure and a set of subplots
fig, ax = plt.subplots(1,1,figsize=(64,36))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False) 
xlist = np.linspace(-0.3, 0.55, 32)
ylist = np.linspace(-0.5, 0.35, 32)

Y, X= np.meshgrid(ylist, xlist)

# Method to change the contour data points
def animate(index):
    ax.clear()
    ax.set_ylim([0.55,-0.3])
    cp = ax.contourf(Y, X, Z_mp[index].min(axis=(0,3)))
    if index<30:
        rect_loc = 0.10
        rect_size = 0.075
        circle_loc = 0 + 0.30/29 * index
    elif index<60:
        rect_loc = 0.10 - 0.15 * (index-30)/29.0
        rect_size = 0.075 + 0.30 * (index-30)/29.0
        circle_loc = 0.30
    else:
        rect_loc = -0.05
        rect_size = 0.375
        circle_loc = 0.30 - 0.30/29 * (index-60)        
    rect = patches.Rectangle((-0.18, rect_loc), 0.16, rect_size, linewidth=1, edgecolor='black',lw=20, facecolor='chocolate')
    circle1 = plt.Circle((0.28, circle_loc), 0.06,color='black',fill=False,lw=100)
    circle2 = plt.Circle((0.28, circle_loc), 0.06,color='gray',fill=False,lw=60)
    ax.add_patch(rect)
    ax.add_patch(circle1)
    ax.add_patch(circle2)
    ax.set_xlabel(str(index))
    if i == 0:
        fig.colorbar(cp,ax=ax) # Add a colorbar to a plot

# Call animate method
ani = animation.FuncAnimation(fig, animate, len(Z_mp), interval=50, blit=False,repeat_delay=1000)
ani.save("exp3_mp.mp4", writer=writer)

## Side-by-side Scene-motion View

Need to create folder directory outputs/exp3/

In [ ]:
grid_size = 9
grid_size_ = 1.0 * (grid_size-1)

corners ={
    0 : (0,0),
    2 : (0,grid_size-1),
    6 : (grid_size-1,0),
    8 : (grid_size-1,grid_size-1),
}   
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial
xlist = np.linspace(-0.5, 0.5, 256)
ylist = np.linspace(-0.5, 0.5, 144)
X, Y = np.meshgrid(xlist, ylist)

fig, ax = plt.subplots()
line1, = ax.plot([], [], 'ro')
Z_rgb = list()

corners =[0,2,6,8]

def get_interp_embedding(x,y,embeddings):
    R1 = embeddings[0]*(grid_size_-x)/grid_size_ + embeddings[1]*x/grid_size_
    R2 = embeddings[2]*(grid_size_-x)/grid_size_ + embeddings[3]*x/grid_size_
    P = R1 * (grid_size_-y)/grid_size_ + R2 * y/grid_size_
    return P

for axis1 in range(grid_size):
    for axis2 in range(grid_size):
        with torch.no_grad():
            samples = torch.from_numpy(np.vstack([Y.reshape(-1),X.reshape(-1)])).float().permute(1,0).cuda()            
            samples.requires_grad=False
            embeddings = list()
            weights = list()            
            for corner in corners: 
                subject_idx = torch.Tensor([corner]).squeeze().long().cuda()[None,...]
                embeddings.append(model.latent_codes(subject_idx))      

            embedding = get_interp_embedding(axis1,axis2,embeddings)
            out = model.inference(samples,embedding,epoch=1000)['rgb'].squeeze().detach().cpu().numpy()+0.5
            out[out>1]=1
            out[out<0]=0
            Z_rgb.append(out.reshape(144,256,3))
            
xlist = np.linspace(-0.3, 0.55, 32)
ylist = np.linspace(-0.5, 0.35, 32)
zlist = np.linspace(0, 0.4, 32)
tlist = np.linspace(-0.5, 0.5, 32)      
X,T,   Y, Z = np.meshgrid(xlist,tlist,ylist,zlist)
Z_mp = list()

for axis1 in range(grid_size):
    for axis2 in range(grid_size):
        with torch.no_grad(): 
            embeddings = list()
            weights = list()            
            for corner in corners: 
                subject_idx = torch.Tensor([corner]).squeeze().long().cuda()[None,...]
                embeddings.append(model.latent_codes(subject_idx))      

            embedding = get_interp_embedding(axis1,axis2,embeddings)
            
            samples_mp = torch.from_numpy(np.vstack([T.reshape(-1),X.reshape(-1),Y.reshape(-1),Z.reshape(-1)])).float().permute(1,0).cuda()
            samples_mp.requires_grad=False
            out = model.inference_mp(samples_mp,embedding,epoch=1000)  
            z_mp = out['mp'].squeeze().detach().cpu().numpy() 
            z_mp[z_mp>0.9]=0.9
            z_mp[z_mp<0]=0
            z_mp = z_mp.reshape(32,32,32,32)
            Z_mp.append(z_mp)

In [ ]:
# Creating bilinear_interp_images
# import os
# os.mkdir("outputs")
# os.mkdir("outputs/exp3")

fig, ax = plt.subplots(1,1,figsize=(64,36))
fig2, ax2 = plt.subplots(1,1,figsize=(64,36))
xlist = np.linspace(-0.3, 0.55, 32)
ylist = np.linspace(-0.5, 0.35, 32)

Y, X= np.meshgrid(ylist, xlist)
def draw(index):
    ax.clear()    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False) 
    ax.imshow(Z_rgb[index]) 
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False) 
    ax2.set_ylim([0.55,-0.3])
    wall_size=index%grid_size
    bottle_location = index//grid_size
    
    cp = ax2.contourf(Y, X, Z_mp[index].min(axis=(0,3)),levels=[0,0.2,0.4,0.6,0.8,1])
    circle_loc = 0 + 0.30/grid_size_ * bottle_location
    rect_loc = 0.10 - 0.15 * wall_size/grid_size_
    rect_size = 0.075 + 0.30 * wall_size/grid_size_

    rect = patches.Rectangle((-0.18, rect_loc), 0.16, rect_size, linewidth=1, edgecolor='black',lw=20, facecolor='chocolate')
    circle1 = plt.Circle((0.28, circle_loc), 0.06,color='black',fill=False,lw=100)
    circle2 = plt.Circle((0.28, circle_loc), 0.06,color='gray',fill=False,lw=60)
    ax2.add_patch(rect)
    ax2.add_patch(circle1)
    ax2.add_patch(circle2)
    fig.savefig('outputs/exp3/rgb_'+str(index)+'.png',bbox_inches='tight')   # save the figure to file
    fig2.savefig('outputs/exp3/mp_'+str(index)+'.png',bbox_inches='tight')   # save the figure to file
for index in range(grid_size**2):
    draw(index)

In [ ]:
import cv2
for index in range(grid_size**2):
    # read the images
    img1 = cv2.imread('outputs/exp3/rgb_'+str(index)+'.png')
    img2 = cv2.imread('outputs/exp3/mp_'+str(index)+'.png')
    im_h = cv2.hconcat([img1, img2])
    cv2.imwrite('outputs/exp3/'+str(index)+'.png', im_h)